### Create Street Intersection Point Feature Class 
#### Projects from FL North to WGS84
- Requires a line featureclass with street names
- ```FeatureVerticesToPoints_management()``` [Available with Advanced license](http://pro.arcgis.com/en/pro-app/tool-reference/data-management/feature-vertices-to-points.htm)

#### Using [FeatureVerticesToPoints_management()](http://pro.arcgis.com/en/pro-app/tool-reference/data-management/feature-vertices-to-points.htm) will create a point at ```BOTH_ENDS``` for each line representing the street centerline
#### [FindIdentical_management()](http://pro.arcgis.com/en/pro-app/tool-reference/data-management/find-identical.htm) will create the table from the point featureclass created above. The identical ```fields``` is simply the SHAPE of the point created where one street segment ends and another street segment begins.

Known Issue: Looping through the list created will exlclude points that are not representing the intersection of a streets with different names

New Capability: Added field to support hyperlinks to Google Streetview (and next Bing BirdsEye)

TODO: Use Data Frame Merge so State Plane FL North feature class will have WGS84 Bing Birds Eye hyperlinks

TODO: Explore [Spatially Enabled Data Frame](https://developers.arcgis.com/python/guide/introduction-to-the-spatially-enabled-dataframe/)

TODO: Add GlobalIDs in source Enterprise Geodatabase

In [ ]:
import arcpy
import numpy as np
import pandas as pd

In [ ]:
sourceStreets = r'C:\GISData\Data\Snapshot\mxBaseMap.geodatabase\main.Cartographic\main.Streets'
destGDB = r'C:\Users\friendde\Documents\ArcGIS\Projects\NAStreets\NAStreets.gdb'
stIntersection = r'C:\Users\friendde\Documents\ArcGIS\Projects\NAStreets\NAStreets.gdb\StreetIntersection'
stIntersectionWGS84 = r'C:\Users\friendde\Documents\ArcGIS\Projects\NAStreets\NAStreets.gdb\StreetIntersectionWGS84'
stVertices = r'C:\Users\friendde\Documents\ArcGIS\Projects\NAStreets\NAStreets.gdb\StreetVertices'
identEnds = r'C:\Users\friendde\Documents\ArcGIS\Projects\NAStreets\NAStreets.gdb\IdenticalStreetEnds'
fldNames = {'IntersectingStreets':'Intersecting Streets','StreetName1':'Street Name 1','StreetName2':'Street Name 2','StreetName3':'Street Name 3','StreetName4':'Street Name 4','Streetview':'Google Streetview'}
outFlds =['IntersectingStreets','StreetName1','StreetName2','StreetName3','StreetName4','POINT_X','POINT_Y','Streetview','GlobalID']
outFldsWGS84 =['IntersectingStreets','StreetName1','StreetName2','StreetName3','StreetName4','POINT_X','POINT_Y','GlobalID']
outFldsGoogle =['IntersectingStreets','StreetName1','StreetName2','StreetName3','StreetName4','POINT_X','POINT_Y','Streetview','GlobalID']
outDirectory = 'C://Users//friendde//Documents//ArcGIS//Projects//NAStreets//'

Cleanup Workspace - caution deletes all existing objects in destGDB, ie feature classes and tables

In [ ]:
arcpy.env.workspace = destGDB
for fc in arcpy.ListFeatureClasses():
    arcpy.Delete_management(fc)
for tbl in arcpy.ListTables():
    arcpy.Delete_management(tbl)

Create the Street Intersection Point Feature Class

In [ ]:
sr = arcpy.Describe(sourceStreets).spatialReference
arcpy.CreateFeatureclass_management(destGDB,'StreetIntersection','POINT',spatial_reference=sr,out_alias='Steet Intersection')
for fName,fAlias in fldNames.items():
    #print(fName,fAlias)
    arcpy.AddField_management (stIntersection,fName,'TEXT',field_length=100,field_alias=fAlias,)

Create intersection points by finding vertices where two or more streets intersect

In [ ]:
arcpy.FeatureVerticesToPoints_management(sourceStreets,stVertices,'BOTH_ENDS')
arcpy.FindIdentical_management(stVertices,identEnds,'SHAPE',output_record_option='ONLY_DUPLICATES')
lastFeatSeq = [row for row in arcpy.da.SearchCursor(identEnds, "FEAT_SEQ")][-1]
lastFeatSeq = lastFeatSeq[0]+1
print(f"number of intersections: {lastFeatSeq}")

Split street intersection names into individual fields for use in MIMS Mobile Search Tool

In [ ]:
for i in range(1,lastFeatSeq,1):
    FID = []
    streetIntersection = []
    with arcpy.da.SearchCursor(identEnds,["IN_FID","FEAT_SEQ"],f"FEAT_SEQ = {i}") as sc:
        for fid in sc:
            FID.append(fid[0])
        for oid in FID:
            with arcpy.da.SearchCursor(stVertices,["OID@","Street_Label","SHAPE@"],f'OBJECTID = {oid}') as stCur:
                for st in stCur:
                    if len(st[1])>2:
                        streetIntersection.append(st[1])
        # Convert list to set and then back to list to remove duplicate street label names
        streetIntersect = list(set(streetIntersection))
        streetIntersect.sort()
        #print(f"sorted list {streetIntersect} length {len(streetIntersect)}")
        # convert list to string
        stringIntersect = '<->'.join(streetIntersect)
        if len(streetIntersect) == 2:
            #print(f"{stringIntersect}")
            ic = arcpy.da.InsertCursor(stIntersection,["IntersectingStreets","StreetName1","StreetName2","SHAPE@"])
            row = [stringIntersect,streetIntersect[0],streetIntersect[1],st[2]]
            ic.insertRow(row)
            del ic
        elif len(streetIntersect) == 3:
            #print(f"{stringIntersect}")
            ic = arcpy.da.InsertCursor(stIntersection,["IntersectingStreets","StreetName1","StreetName2","StreetName3","SHAPE@"])
            row = [stringIntersect,streetIntersect[0],streetIntersect[1],streetIntersect[2],st[2]]
            ic.insertRow(row)
            del ic
        elif len(streetIntersect) == 4:
            #print(f"{stringIntersect}")
            ic = arcpy.da.InsertCursor(stIntersection,["IntersectingStreets","StreetName1","StreetName2","StreetName3","StreetName4","SHAPE@"])
            row = [stringIntersect,streetIntersect[0],streetIntersect[1],streetIntersect[2],streetIntersect[3],st[2]]
            ic.insertRow(row)
            del ic
        else:
            pass
            #print(f"string list other {stringIntersect}")
            #ic = arcpy.da.InsertCursor(stIntersection,["IntersectingStreets","SHAPE@"])
            #row = [stringIntersect,st[2]]
            #ic.insertRow(row)
            #del ic

Count number of features for reference

In [ ]:
lastIntersect = [row for row in arcpy.da.SearchCursor(stIntersection, "OID@")][-1]
print(f"number of intersections: {lastIntersect}")

In [ ]:
#Add GUID to make use of in Data Frame Join 
arcpy.AddGlobalIDs_management(stIntersection)

In [ ]:
#Project to WGS84 to obtain Lat Long during AddXY
srWGS84 = arcpy.SpatialReference(4326)
arcpy.Project_management(stIntersection, stIntersectionWGS84, srWGS84)
arcpy.AddXY_management(stIntersectionWGS84)

Perform this step after Project so we will have State Plane FL North Coords in original stIntersection

In [ ]:
# AddXY to original stIntersection for use later
#TODO will new Spatially Enabled Data Frame negate this step?
arcpy.AddXY_management(stIntersection)

Using [arcpy.da.TableToNumPyArray()](http://pro.arcgis.com/en/pro-app/arcpy/data-access/tabletonumpyarray.htm) and [pandas.DataFrame()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) display the first 5 rows of the Street Intersection point featureclass

In [ ]:
#Create Data Frame with Projected feature clasee and verify results
arrWGS84 = arcpy.da.TableToNumPyArray(stIntersectionWGS84,outFldsWGS84)
dfWGS84 = pd.DataFrame(arrWGS84)
#verify results
dfWGS84.head()

In [ ]:
#Change dtype of X and Y to string
dfWGS84 = dfWGS84.astype({'POINT_X':str}, copy=False)
dfWGS84 = dfWGS84.astype({'POINT_Y':str}, copy=False)
dfWGS84.dtypes

Google Streetview Template - http://maps.google.com/?cbll=%VALUE%&cbp=12,90,0,0,5&layer=c

Bing Birdseye Template - 

Note cbp parameter, if you want default street view to point a certain compass direction alter second number

In [ ]:
#default view towards 90 degrees
#df = df.assign(StreetView='http://maps.google.com/?cbll=' + df["POINY_Y"] + df["POINT_X"] + '&cbp=12,90,0,0,5&layer=c')
# default view towards 0 degrees
dfWGS84 = dfWGS84.assign(Google='http://maps.google.com/?cbll=' + dfWGS84.POINT_Y + ',' + dfWGS84.POINT_X + '&cbp=12,0,0,0,5&layer=c')
# verify Google Streetview results
dfWGS84.Google.head()

In [ ]:
#Create Data Frame for original stIntersection and prep for Data Frame Merge
arr = arcpy.da.TableToNumPyArray(stIntersection,outFlds)
df = pd.DataFrame(arr)
df.head()

In [ ]:
# add BaseURL to orginal stIntersection df for use in MIMS Configuration
df = df.assign(BaseURLGoogle= dfWGS84.POINT_Y + ',' + dfWGS84.POINT_X)
# verify Google Streetview results
df.BaseURLGoogle.head()

In [ ]:
# Drop Pandas Data Frame Columns in dfWGS84 to prep for Data Frame Merge
dfWGS84 = dfWGS84.drop(['IntersectingStreets','StreetName1','StreetName2','StreetName3','StreetName4','POINT_X','POINT_Y'],axis=1)
#verify results
dfWGS84.head()

In [ ]:
#Merge the two Data Frames
df2 = pd.merge(df,dfWGS84, how='outer', on='GlobalID', left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=False, indicator=False, validate=None)
#verify results
df2.head()

In [ ]:
#Update Streetview Hyperlink field that original stIntersection wull use in MIMS Mobile
df2.Streetview = df2.Google
#verify results
df2.head()

In [ ]:
#overwrite orginal df and drop extra columns from previous Merge
df = df2.drop(['Google'], axis = 1)
#verify results
df.head()

The following steps my not be required when using new Spatially Enabled Data Frame

In [ ]:
df.to_csv(outDirectory + 'StreetIntersection.csv',header=True, index=False)
arcpy.env.workspace = destGDB
arcpy.env.overwriteOutput = True
#arcpy.TableToTable_conversion (outDirectory + 'StreetIntersection.csv', destGDB, 'StreetIntersection')
arcpy.XYTableToPoint_management (outDirectory + 'StreetIntersection.csv', 'StreetIntersection', 'POINT_X', 'POINT_Y','',sr)
arcpy.env.workspace = outDirectory
arcpy.env.overwriteOutput = True
arcpy.ExportXMLWorkspaceDocument_management (stIntersection, outDirectory + 'StreetIntersection.xml', 'DATA')